In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import linear_model

In [ ]:
data_all = np.genfromtxt('../data/kc_house_data.csv', dtype=None, delimiter=',', names=True)
data = np.genfromtxt('../data/wk3_kc_house_train_data.csv', dtype=None, delimiter=',', names=True)
d_val = np.genfromtxt('../data/wk3_kc_house_valid_data.csv', dtype=None, delimiter=',', names=True)
d_test = np.genfromtxt('../data/wk3_kc_house_test_data.csv', dtype=None, delimiter=',', names=True)

In [ ]:
def normalize_features(X):
    norms = np.linalg.norm(X, axis=0)
    X_normalized = X / norms
    return X_normalized, norms


def predict_outcome(feature_matrix, weights):
    return np.dot(feature_matrix, weights)


def compute_ro(weights, feature, output, predictions):
    assert(weights.shape[0] == feature.shape[1])

    ro = np.empty((weights.shape[0]))
    diff = output - predictions
    for i in range(weights.shape[0]):
        ro[i] = np.sum(np.multiply(feature[:, i], diff + np.dot(feature[:, i], weights[i])))
    
    return ro

In [ ]:
inp = np.array([data_all['sqft_living'], data_all['bedrooms']]).T
inp = np.array([np.ones_like(inp[:, 0]), inp[:, 0], inp[:, 1]]).T

output = data_all['price']

In [ ]:
inp_norm, norms = normalize_features(inp)

initial_weights = np.array([1., 4., 1.])

In [ ]:
init_pred = predict_outcome(inp_norm, initial_weights)

ro = compute_ro(initial_weights, inp_norm, output, init_pred)

print(ro)
# quiz question (w[1] -> not zero, w[2] -> zero): we want l1_penalty to follow the inequality: 
# 2 * abs(ro[2]) < l1_penalty < 2 * abs(ro[1])
# quiz question (w[1] -> zero, w[2] -> zero): l1_penalty < 2 * max(ro[1], ro[2])
print('1: {} < l1 < {},   2: l1_pen > {}'.format(2 * ro[2], 2 * ro[1], 2 * max(ro[1], ro[2])))

In [ ]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    weights = np.array(weights)
    # compute prediction
    prediction = predict_outcome(feature_matrix, weights)
    diff = output - prediction
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = np.sum(np.multiply(feature_matrix[:, i], diff + np.dot(feature_matrix[:, i], weights[i])))
    
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty / 2.
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty / 2.
    else:
        new_weight_i = 0.
    
    return new_weight_i

In [ ]:
# should not raise assertion (or any other) error
import math
assert(abs(lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],
                   [2./math.sqrt(13),3./math.sqrt(10)]]), np.array([1., 1.]), np.array([1., 4.]), 0.1) - 0.425558846) < 0.0001) 

In [ ]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    weights = np.copy(initial_weights)
    converged = False
    while not converged:
        for i in range(initial_weights.shape[0]):
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)

        # check whether it has converged (max of difference is below tolerance)
        converged = np.abs(np.max(weights - initial_weights)) < tolerance
        initial_weights = np.copy(weights)
        
    return weights


def compute_rss_lasso(y_pred, y_true, w):
    erri = y_pred - y_true
    return np.sum(np.multiply(erri, erri)) + np.linalg.norm(w, 1)

In [ ]:
in_w = np.array([0., 0., 0.])
l1_penalty = 1e7
tolerance = 1.0
weights1 = lasso_cyclical_coordinate_descent(inp_norm, output, in_w, l1_penalty, tolerance)

In [ ]:
y_pred = predict_outcome(inp_norm, weights1)
print(compute_rss_lasso(y_pred, output, weights1[1:]))
print(weights1)

In [ ]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']


In [ ]:
def format_feats(data, all_features):
    # hack due to possibly mistaken encoding of floors in the csv files
    tmp = np.empty((len(data['floors'])), dtype=np.float64)
    for i in range(len(data['floors'])):
        tmp[i] = np.float(data['floors'][i][1:-1])
        # [1:-1] -> for some reason they are encoded as "[num]" and cannot 
        # be converted with an obvious way otherwise.
    
    inp_tr = [np.ones_like(tmp)]
    for ft in all_features:
        if ft == 'floors':
            d1 = tmp
        else:
            d1 = data[ft]
        assert(not np.any(np.isnan(d1)))
        inp_tr.append(d1)

    inp_tr = np.array(inp_tr).T
    return inp_tr


def aux_print_nonzero_weights(weights, all_features):
    idx = np.where(weights != 0.)
    for i in idx[0]:
        if i == 0:
            print('intercept')
            continue
        print(all_features[i - 1])

In [ ]:
inp_tr = format_feats(data, all_features)
output = data['price']
inp_tr_n, norms_tr = normalize_features(inp_tr)

# initial weights
in_w = np.zeros((inp_tr.shape[1]))

In [ ]:
weights1e7 = lasso_cyclical_coordinate_descent(inp_tr_n, output, in_w, l1_penalty, tolerance)
print('Weights with l1_pen = {}: \n{}'.format(l1_penalty, weights1e7))
aux_print_nonzero_weights(weights1e7, all_features)

In [ ]:
in_w = np.zeros((inp_tr.shape[1]))
l1_penalty = 1e8
weights1e8 = lasso_cyclical_coordinate_descent(inp_tr_n, output, in_w, l1_penalty, tolerance)
print('Weights with l1_pen = {}: \n{}'.format(l1_penalty, weights1e8))
aux_print_nonzero_weights(weights1e8, all_features)

In [ ]:
in_w = np.zeros((inp_tr.shape[1]))
l1_penalty = 1e4
weights1e4 = lasso_cyclical_coordinate_descent(inp_tr_n, output, in_w, l1_penalty, 5e5)
print('Weights with l1_pen = {}: \n{}'.format(l1_penalty, weights1e4))
aux_print_nonzero_weights(weights1e4, all_features)

# Normalise the weights learnt

In [ ]:
normalized_weights1e7 = weights1e7 / norms_tr
print normalized_weights1e7
normalized_weights1e8 = weights1e8 / norms_tr
normalized_weights1e4 = weights1e4 / norms_tr

In [ ]:
# test on testset
inp_t = format_feats(d_test, all_features)
assert(inp_tr.shape[1] ==  inp_t.shape[1])  # should have the same feats as in training
out_t = d_test['price']

In [ ]:
y_pred = predict_outcome(inp_t, normalized_weights1e7)
print(compute_rss_lasso(y_pred, out_t, normalized_weights1e7[1:]))

In [ ]:
y_pred = predict_outcome(inp_t, normalized_weights1e8)
print(compute_rss_lasso(y_pred, out_t, normalized_weights1e8[1:]))

In [ ]:
y_pred = predict_outcome(inp_t, normalized_weights1e4)
print(compute_rss_lasso(y_pred, out_t, normalized_weights1e4[1:]))